## Predicting star ratings of a restaurant review

In [1]:
import pandas as pd
import string
import nltk
import numpy as np
import keras
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, TimeDistributed, Bidirectional
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Input
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
#Save checkpoint
# data.to_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint", header=True, index=False)

#Use if kernel crashes
data = pd.read_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint")
data.head()

,index,user,item,review,rating,real,date
0,0,5044,0,drink be bad the hot chocolate be water down a...,1.0,-1,2014-11-16
1,1,5045,0,this be the bad experience ive ever have a cas...,1.0,-1,2014-09-08
2,2,5046,0,this be locate on the site of the old spruce s...,3.0,-1,2013-10-06
3,3,5047,0,i enjoy coffee and breakfast twice at toast du...,5.0,-1,2014-11-30
4,4,5048,0,i love toast the food choice be fantastic i lo...,5.0,-1,2014-08-28


In [3]:
data['review'] = data['review'].apply(lambda s: str(s))

In [4]:
ratings = pd.get_dummies(data['rating'])
ratings

,1.0,2.0,3.0,4.0,5.0
0,1,0,0,0,0
1,1,0,0,0,0
2,0,0,1,0,0
3,0,0,0,0,1
4,0,0,0,0,1
5,0,0,0,0,1
6,0,0,0,0,1
7,1,0,0,0,0
8,0,1,0,0,0
9,0,0,0,1,0


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['review'], ratings, test_size=0.30, random_state=42)

In [6]:
tokenizer = Tokenizer(num_words= 4000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
train_sequence = pad_sequences(sequences, maxlen=150)

In [7]:
ip = Input(shape=[150])
lyr = Embedding(4000, 32, input_length=150)(ip)
lyr = LSTM(256, recurrent_dropout=0.2)(lyr)
lyr = Dense(256)(lyr)
lyr = Dropout(0.3)(lyr)
lyr = Dense(5)(lyr)
lyr = Activation('softmax')(lyr)
model = Model(inputs=ip, outputs=lyr)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 32)           128000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               295936    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total para

In [10]:
model.fit(train_sequence, y_train, batch_size=500, epochs=1, verbose = 1)

Epoch 1/1


102000/425920 [======>.......................] - ETA: 1:00:15 - loss: 0.5004 - acc: 0.80 - ETA: 44:41 - loss: 0.4986 - acc: 0.8000 - ETA: 39:29 - loss: 0.4968 - acc: 0.80 - ETA: 36:28 - loss: 0.4945 - acc: 0.80 - ETA: 35:08 - loss: 0.4906 - acc: 0.80 - ETA: 33:50 - loss: 0.4848 - acc: 0.80 - ETA: 33:10 - loss: 0.4824 - acc: 0.79 - ETA: 32:27 - loss: 0.4776 - acc: 0.79 - ETA: 31:53 - loss: 0.4746 - acc: 0.79 - ETA: 31:40 - loss: 0.4720 - acc: 0.79 - ETA: 31:14 - loss: 0.4697 - acc: 0.79 - ETA: 30:54 - loss: 0.4677 - acc: 0.79 - ETA: 30:47 - loss: 0.4647 - acc: 0.79 - ETA: 30:34 - loss: 0.4625 - acc: 0.79 - ETA: 30:28 - loss: 0.4613 - acc: 0.79 - ETA: 30:16 - loss: 0.4598 - acc: 0.79 - ETA: 30:03 - loss: 0.4585 - acc: 0.79 - ETA: 30:01 - loss: 0.4575 - acc: 0.79 - ETA: 29:52 - loss: 0.4563 - acc: 0.79 - ETA: 29:44 - loss: 0.4554 - acc: 0.79 - ETA: 29:43 - loss: 0.4552 - acc: 0.79 - ETA: 29:36 - loss: 0.4548 - acc: 0.79 - ETA: 29:34 - loss: 0.4539 - acc: 0.79 - ETA: 29:28 - loss: 0.4531 -

306000/425920 [====================>.........] - ETA: 18:41 - loss: 0.3693 - acc: 0.82 - ETA: 18:39 - loss: 0.3693 - acc: 0.82 - ETA: 18:37 - loss: 0.3692 - acc: 0.82 - ETA: 18:34 - loss: 0.3691 - acc: 0.82 - ETA: 18:32 - loss: 0.3691 - acc: 0.82 - ETA: 18:29 - loss: 0.3690 - acc: 0.82 - ETA: 18:27 - loss: 0.3690 - acc: 0.82 - ETA: 18:25 - loss: 0.3689 - acc: 0.82 - ETA: 18:22 - loss: 0.3689 - acc: 0.82 - ETA: 18:20 - loss: 0.3688 - acc: 0.82 - ETA: 18:17 - loss: 0.3688 - acc: 0.82 - ETA: 18:15 - loss: 0.3687 - acc: 0.82 - ETA: 18:13 - loss: 0.3687 - acc: 0.82 - ETA: 18:10 - loss: 0.3686 - acc: 0.82 - ETA: 18:08 - loss: 0.3685 - acc: 0.82 - ETA: 18:05 - loss: 0.3684 - acc: 0.82 - ETA: 18:03 - loss: 0.3683 - acc: 0.82 - ETA: 18:01 - loss: 0.3682 - acc: 0.82 - ETA: 17:58 - loss: 0.3682 - acc: 0.82 - ETA: 17:56 - loss: 0.3681 - acc: 0.82 - ETA: 17:54 - loss: 0.3680 - acc: 0.82 - ETA: 17:51 - loss: 0.3680 - acc: 0.82 - ETA: 17:49 - loss: 0.3679 - acc: 0.82 - ETA: 17:46 - loss: 0.3678 - acc

408000/425920 [===========================>..] - ETA: 10:24 - loss: 0.3583 - acc: 0.82 - ETA: 10:22 - loss: 0.3582 - acc: 0.82 - ETA: 10:19 - loss: 0.3582 - acc: 0.82 - ETA: 10:17 - loss: 0.3582 - acc: 0.82 - ETA: 10:14 - loss: 0.3581 - acc: 0.82 - ETA: 53:46 - loss: 0.3581 - acc: 0.82 - ETA: 53:29 - loss: 0.3581 - acc: 0.82 - ETA: 53:11 - loss: 0.3580 - acc: 0.82 - ETA: 52:54 - loss: 0.3580 - acc: 0.82 - ETA: 52:36 - loss: 0.3579 - acc: 0.82 - ETA: 52:18 - loss: 0.3579 - acc: 0.82 - ETA: 52:00 - loss: 0.3578 - acc: 0.82 - ETA: 51:43 - loss: 0.3578 - acc: 0.82 - ETA: 51:25 - loss: 0.3578 - acc: 0.82 - ETA: 51:07 - loss: 0.3577 - acc: 0.82 - ETA: 50:50 - loss: 0.3577 - acc: 0.82 - ETA: 50:32 - loss: 0.3577 - acc: 0.82 - ETA: 50:15 - loss: 0.3576 - acc: 0.82 - ETA: 49:58 - loss: 0.3576 - acc: 0.82 - ETA: 49:40 - loss: 0.3575 - acc: 0.82 - ETA: 49:23 - loss: 0.3575 - acc: 0.82 - ETA: 49:06 - loss: 0.3575 - acc: 0.82 - ETA: 48:48 - loss: 0.3575 - acc: 0.82 - ETA: 48:31 - loss: 0.3575 - acc

425920/425920 [==============================] - ETA: 6:26 - loss: 0.3522 - acc: 0.828 - ETA: 6:15 - loss: 0.3522 - acc: 0.828 - ETA: 6:03 - loss: 0.3521 - acc: 0.828 - ETA: 5:52 - loss: 0.3521 - acc: 0.828 - ETA: 5:41 - loss: 0.3521 - acc: 0.828 - ETA: 5:29 - loss: 0.3521 - acc: 0.828 - ETA: 5:18 - loss: 0.3521 - acc: 0.828 - ETA: 5:07 - loss: 0.3520 - acc: 0.828 - ETA: 4:55 - loss: 0.3520 - acc: 0.828 - ETA: 4:44 - loss: 0.3520 - acc: 0.828 - ETA: 4:33 - loss: 0.3520 - acc: 0.828 - ETA: 4:22 - loss: 0.3519 - acc: 0.828 - ETA: 4:10 - loss: 0.3519 - acc: 0.828 - ETA: 3:59 - loss: 0.3519 - acc: 0.828 - ETA: 3:48 - loss: 0.3519 - acc: 0.828 - ETA: 3:37 - loss: 0.3518 - acc: 0.828 - ETA: 3:26 - loss: 0.3518 - acc: 0.828 - ETA: 3:15 - loss: 0.3518 - acc: 0.828 - ETA: 3:03 - loss: 0.3518 - acc: 0.828 - ETA: 2:52 - loss: 0.3518 - acc: 0.828 - ETA: 2:41 - loss: 0.3517 - acc: 0.828 - ETA: 2:30 - loss: 0.3517 - acc: 0.828 - ETA: 2:19 - loss: 0.3517 - acc: 0.828 - ETA: 2:08 - loss: 0.3517 - acc:

In [12]:
test_seq = tokenizer.texts_to_sequences(X_test)
test_sequence = pad_sequences(test_seq, maxlen=100)

In [13]:
y_pred = model.predict_classes(test_sequence)

In [16]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pd.get_dummies(y_pred)))

              precision    recall  f1-score   support

           0       0.49      0.07      0.12     10615
           1       0.26      0.09      0.13     12979
           2       0.33      0.23      0.27     24707
           3       0.42      0.74      0.53     65318
           4       0.62      0.39      0.48     68919

   micro avg       0.45      0.45      0.45    182538
   macro avg       0.42      0.30      0.31    182538
weighted avg       0.47      0.45      0.42    182538
 samples avg       0.45      0.45      0.45    182538



In [15]:
results = model.evaluate(test_sequence, y_test)

 25920/182538 [===>..........................] - ETA: 1:05:2 - ETA: 36:22  - ETA: 26:4 - ETA: 21:5 - ETA: 18:5 - ETA: 17:0 - ETA: 15:4 - ETA: 14:4 - ETA: 14:1 - ETA: 13:3 - ETA: 13:0 - ETA: 12:3 - ETA: 12:1 - ETA: 11:5 - ETA: 11:3 - ETA: 11:1 - ETA: 11:0 - ETA: 10:5 - ETA: 10:4 - ETA: 10:2 - ETA: 10:1 - ETA: 10:0 - ETA: 10:0 - ETA: 9:5 - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 9: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 8: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - 

 52000/182538 [=======>......................] - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - ETA: 7: - E

 78080/182538 [===========>..................] - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - ETA: 6: - E

104160/182538 [================>.............] - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - ETA: 4: - E

130240/182538 [====================>.........] - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - ETA: 3: - E

156320/182538 [========================>.....] - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - ETA: 2: - E

182538/182538 [==============================] - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - ETA: 1: - E

In [16]:
print("Accuracy:", results[1]*100, "%")

Accuracy: 83.58500641898978 %


In [37]:
#Saving the model
model.save(r'C:\Users\elonm\Desktop\MachineLearningProject\good_bad_model.h5')

In [8]:
#Loading model
from keras.models import load_model
model = load_model('good_bad_model.h5')

In [ ]:
model.evaluate()

In [36]:
test_str = input("Enter a test string:")
test_str = test_str.lower()
tst_seq = tokenizer.texts_to_sequences([test_str])
tst_s = pad_sequences(tst_seq, maxlen=150)
tst_res = model.predict(tst_s)
print(tst_res)
print("Stars:", str(np.argmax(tst_res)+1))

Enter a test string:The venue was fine. Nothing special
[[0.08844485 0.1718511  0.3160937  0.27570808 0.14790224]]
Stars: 3
